# Comparison Statistics between Cohorts

## Python Setup

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from pandas import Series
import glob
from openpyxl import load_workbook
import re
pd.options.display.max_columns = 100

## Load In Data

In [2]:
funded_grnts = pd.read_csv('../data/funded_grants.csv', low_memory=False)
grnts = pd.read_csv('../data/researcher_grants.csv', low_memory=False)
pubs = pd.read_csv('../data/researcher_publications.csv', low_memory=False)
rsrs = pd.read_csv('../data/researcher_info.csv', low_memory=False)
propensity_scores = pd.read_csv('../data/rsr_weights.csv', low_memory=False)

In [3]:
grnts_ct = pd.read_csv('../data/topic_lookups/grants_cancer_type.csv')
grnts_cso = pd.read_csv('../data/topic_lookups/grants_cso.csv')
grnts_rcdc = pd.read_csv('../data/topic_lookups/grants_rcdc.csv')
pubs_ct = pd.read_csv('../data/topic_lookups/publications_cancer_type.csv')
pubs_cso = pd.read_csv('../data/topic_lookups/publications_cso.csv')
pubs_rcdc = pd.read_csv('../data/topic_lookups/publications_rcdc.csv')

cso_lookup = pd.read_csv('../data/cso_codes/cso_lookup.csv')

In [4]:
print(funded_grnts[funded_grnts['rsr_id'].isnull()].shape)
print(grnts[grnts['rsr_id'].isnull()].shape)
print(pubs[pubs['pub_id'].isnull()].shape)
print(pubs[pubs['date'].isnull()].shape)

(0, 8)
(0, 6)
(0, 17)
(163, 17)


In [5]:
funded_grnts = funded_grnts[(funded_grnts['rsr_id'].notnull())
                            &(funded_grnts['grant_id'].notnull())].reset_index(drop=True)
grnts = grnts[(grnts['rsr_id'].notnull())
              &(grnts['grant_id'].notnull())].reset_index(drop=True)
pubs = pubs[(pubs['pub_id'].notnull())&(pubs['date'].notnull())].reset_index(drop=True)

In [6]:
funders = sorted(list(funded_grnts['funder_name'].unique()))
print(funders)

['Cancer Research UK', 'INCa/INSERM/DGOS', 'National Cancer Institute', 'National Health and Medical Research Council', 'Wellcome Trust']


## Cleaning Data

In [7]:
pubs['citations'] = pd.to_numeric(pubs['citations'])
pubs['nb_authors'] = pd.to_numeric(pubs['nb_authors'], errors='coerce')

In [8]:
funded_grnts['start_date'] = pd.to_datetime(funded_grnts['start_date'])
funded_grnts['end_date'] = pd.to_datetime(funded_grnts['end_date'])
grnts['start_date'] = pd.to_datetime(grnts['start_date'])
grnts['end_date'] = pd.to_datetime(grnts['end_date'])
funded_grnts['funding_len'] = (funded_grnts['end_date']-funded_grnts['start_date'])/timedelta(days=365)
grnts['funding_len'] = (grnts['end_date']-grnts['start_date'])/timedelta(days=365)
funded_grnts['start_year'] = pd.DatetimeIndex(funded_grnts['start_date']).year
grnts['start_year'] = pd.DatetimeIndex(grnts['start_date']).year

In [9]:
funded_grnts['nb_rsrs'] = funded_grnts.groupby('grant_id')['rsr_id'].transform('nunique')
funded_grnts['nb_rsrs'] = funded_grnts['nb_rsrs'].replace(0, np.nan)
grnts['nb_rsrs'] = grnts.groupby('grant_id')['rsr_id'].transform('nunique')
grnts['nb_rsrs'] = grnts['nb_rsrs'].replace(0, np.nan)

In [10]:
grnts_rcdc = grnts_rcdc.drop_duplicates().sort_values('grant_id').reset_index(drop=True)
grnts_rcdc['nb_rcdc'] = grnts_rcdc.groupby('grant_id')['grant_id'].transform('count')

grnts_cso = grnts_cso.drop_duplicates().sort_values('grant_id').reset_index(drop=True)
grnts_cso['nb_cso'] = grnts_cso.groupby('grant_id')['grant_id'].transform('count')

pubs_rcdc = pubs_rcdc.drop_duplicates().sort_values('pub_id').reset_index(drop=True)
pubs_rcdc['nb_rcdc'] = pubs_rcdc.groupby('pub_id')['pub_id'].transform('count')

pubs_cso = pubs_cso.drop_duplicates().sort_values('pub_id').reset_index(drop=True)
pubs_cso['nb_cso'] = pubs_cso.groupby('pub_id')['pub_id'].transform('count')

### Create Lookup Tables

In [11]:
funded_grnts_funder = funded_grnts[['funder_name', 'grant_id']].drop_duplicates().reset_index(drop=True)
grnts_funder = grnts[['funder_name', 'grant_id']].drop_duplicates().reset_index(drop=True)

funded_grnts_rsr = funded_grnts[['rsr_id', 'grant_id']].drop_duplicates().reset_index(drop=True)
grnts_rsr = grnts[['rsr_id', 'grant_id']].drop_duplicates().reset_index(drop=True)

pub_rsr_cols = ['rsr_id', 'rsr_country', 'rsr_city', 'rsr_affiliation', 'rsr_affiliation_id']
pubs_rsr = pubs[pub_rsr_cols+['pub_id']].drop_duplicates().reset_index(drop=True)

In [12]:
to_remove = ('rsr_id', 'funder_name', 'first_name', 'last_name')
cols = [col for col in list(funded_grnts) if col not in to_remove]
funded_grnts_info = funded_grnts[cols].drop_duplicates().reset_index(drop=True)

In [13]:
to_remove = ('rsr_id', 'funder_name')
cols = [col for col in list(grnts) if col not in to_remove]
grnts_info = grnts[cols].drop_duplicates('grant_id').reset_index(drop=True)

In [14]:
cols = [col for col in list(pubs) if col not in pub_rsr_cols]
pubs_info = pubs[cols].drop_duplicates('pub_id').reset_index(drop=True)

### Merging on Prior and Subsequent Grants and Publications

I keep only grants and publications that are within 5 years of the funded grant.

In [15]:
start_year=2007
end_year=2012

col1=[]
col2=[]
col3=[]   
for grant_year in range(start_year, end_year+1):
    for award_year in range(grant_year-5, grant_year+6):
        if grant_year>award_year:
            col1.append(grant_year)
            col2.append("pre")
            col3.append(award_year)
        if grant_year==award_year:
            col1.append(grant_year)
            col2.append("pre")
            col3.append(award_year)
        if grant_year<award_year:
            col1.append(grant_year)
            col2.append("post")
            col3.append(award_year)
merge_key = pd.DataFrame({'start_year':col1, 'status':col2, 'year':col3})

In [16]:
temp = pd.merge(funded_grnts[['funder_name', 'grant_id', 'start_year', 'start_date', 'rsr_id']].drop_duplicates(), 
                merge_key, how='left', on='start_year')

In [17]:
grnt_cols = ['rsr_id', 'grant_id', 'start_year']
grnts_mrg = pd.merge(temp, grnts[grnt_cols].drop_duplicates().rename(columns={'start_year':'year'}),
                     how='left', on=['rsr_id', 'year'], suffixes=('', '_2'))

In [18]:
pub_cols = ['rsr_id', 'pub_id', 'year']
pubs_mrg = pd.merge(temp, pubs[pub_cols].drop_duplicates(),
                    how='left', on=['rsr_id', 'year'], suffixes=('', '_2'))

## Store Results in Dictionary

In [19]:
results = dict()

## Create Statistics Functions

In [20]:
def grant_groupby(grnts_mrg, yearly=False):
    
    grnts_info.columns = [col+"_2" for col in list(grnts_info)]
    df = pd.merge(grnts_mrg, grnts_info.rename(columns={'start_year_2':'year'}), 
                  how='left', on=['grant_id_2', 'year'])
    grnts_info.columns = [col[:-2] for col in list(grnts_info)]

    cols = ['grant_id', 'rsr_id', 'status']
    if yearly == True:
        cols += ['year']
    grnts_stats = df.groupby(cols)
    grnts_stats = pd.DataFrame({'nb_grnts':grnts_stats['grant_id_2'].nunique()
                                , 'fund_amt':grnts_stats['funding_amount_2'].mean()
                                , 'avg_fund_len':grnts_stats['funding_len_2'].mean()
                                , 'avg_team_size':grnts_stats['nb_rsrs_2'].mean()
                               }).reset_index()
    grnts_stats = pd.merge(funded_grnts_funder, grnts_stats, how='left', on='grant_id')
    
    return grnts_stats

In [43]:
def pub_groupby(pubs_mrg, yearly=False):
    
    cols = ['grant_id', 'rsr_id', 'status']
    if yearly == True:
        cols += ['year']
    
    # Publication Statistics
    df = pd.merge(pubs_mrg, pubs_info, how='left', on=['pub_id', 'year'])
    pubs_stats = df.groupby(cols)
    pubs_stats = pd.DataFrame({'nb_pubs':pubs_stats['pub_id'].nunique()
                                , 'citations_per_pub':pubs_stats['citations'].mean()
                                , 'team_size':pubs_stats['nb_authors'].mean()
                               }).reset_index()
    
    # Network Statistics:
    df = pd.merge(pubs_mrg, pubs_rsr.rename(columns={'rsr_id':'author_id'}), 
                  how='left', on='pub_id')
    pubs_collab = df.groupby(cols)
    pubs_collab = pd.DataFrame({'nb_collabs': pubs_collab['author_id'].nunique()
                                , 'nb_collab_countries': pubs_collab['rsr_country'].nunique()
                               }).reset_index()
    
    # In Topic or not in Topic?
    df = pd.merge(pubs_mrg, grnts_rcdc, how='left', on='grant_id')
    df = pd.merge(df, pubs_rcdc, how='left', on='pub_id')
    df['in_topic'] = (df['rcdc_name_x']==df['rcdc_name_y'])
    col_temp = list(pubs_mrg)
    df = df.groupby(col_temp)['in_topic'].sum().reset_index()
    df['in_topic'] = np.where(df['in_topic']>0, True, False)
    pubs_topic = df.groupby(cols)
    pubs_topic = pd.DataFrame({'nb_pubs_in_topic': pubs_topic['in_topic'].sum()
                                , 'pct_pubs_in_topic': pubs_topic['in_topic'].mean()
                               }).reset_index()    
    
    
    # Combine the two:
    pubs_stats = pd.merge(pubs_stats, pubs_collab, how='outer', on=cols)
    pubs_stats = pd.merge(pubs_stats, pubs_topic, how='outer', on=cols)
    pubs_stats = pd.merge(funded_grnts_funder, pubs_stats, how='left', on='grant_id')
    
    return pubs_stats

## 1. Funded Grants Statistics

### Number of Grants per Funder

In [82]:
funded_grnts.describe(include='all')

,funder_name,grant_id,rsr_id,first_name,last_name,start_date,end_date,funding_amount,funding_len,start_year,nb_rsrs
count,22002,22002,22002,21956,22002,22002,21954,1.970800e+04,21954.000000,22002.000000,22002.000000
unique,5,16925,14004,7683,9132,1171,1082,NaN,NaN,NaN,NaN
top,National Cancer Institute,grant.2438876,ur.01117731572.33,David,HEIMBROOK,2009-01-01 00:00:00,2017-01-01 00:00:00,NaN,NaN,NaN,NaN
freq,14682,45,384,595,384,1613,828,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,2007-01-01 00:00:00,2007-01-31 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,2012-12-31 00:00:00,2024-10-31 00:00:00,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.381371e+06,4.247470,2009.450550,2.275520
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.261339e+06,2.682721,1.661078,3.377899
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.002740,2007.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.206820e+05,2.002740,2008.000000,1.000000


In [83]:
results['nb_unique_rsrs'] = {}

print("Number of unique researchers funded:")
print(funded_grnts['rsr_id'].nunique())
results['nb_unique_rsrs']['overall'] = funded_grnts['rsr_id'].nunique()

print("\nNumber of unique researchers funded by each agency:")
print(funded_grnts.groupby('funder_name')['rsr_id'].nunique())
for funder in funders:
    results['nb_unique_rsrs'][funder] = funded_grnts.groupby('funder_name')['rsr_id'].nunique()[funder]

Number of unique researchers funded:
14004

Number of unique researchers funded by each agency:
funder_name
Cancer Research UK                              1418
INCa/INSERM/DGOS                                 981
National Cancer Institute                       9356
National Health and Medical Research Council    2015
Wellcome Trust                                   330
Name: rsr_id, dtype: int64


In [84]:
results['nb_unique_grnts'] = {}

print("Number of unique grants funded:")
print(funded_grnts['grant_id'].nunique())
results['nb_unique_grnts']['overall'] = funded_grnts['grant_id'].nunique()

print("\nNumber of unique grants funded by each agency:")
print(funded_grnts.groupby('funder_name')['grant_id'].nunique())
for funder in funders:
    results['nb_unique_grnts'][funder] = funded_grnts.groupby('funder_name')['grant_id'].nunique()[funder]

Number of unique grants funded:
16925

Number of unique grants funded by each agency:
funder_name
Cancer Research UK                               2253
INCa/INSERM/DGOS                                 1318
National Cancer Institute                       11936
National Health and Medical Research Council     1155
Wellcome Trust                                    263
Name: grant_id, dtype: int64


### Number of Grants per Researcher

In [ ]:
print("Agerage Number of Grants per Researcher:")
print(funded_grnts.groupby('rsr_id')['grant_id'].nunique().mean())
print("\nNumber of grants from agencies per researcher (as % of total):\n")
print(funded_grnts.groupby('rsr_id')['grant_id'].nunique().value_counts(normalize=True).head())

In [ ]:
print("Agerage Number of Agencies per Researcher:")
print(funded_grnts.groupby('rsr_id')['funder_name'].nunique().mean())
print("\nBy how many agencies are the researchers funded?\n")
print(funded_grnts.groupby('rsr_id')['funder_name'].nunique().value_counts(normalize=True))

In [ ]:
print("How many grants does each agency give to its reserachers in the 5 focal years?")
for funder in funders:
    print("\n{}:".format(funder))
    temp = funded_grnts[funded_grnts['funder_name']==funder].groupby('rsr_id')['grant_id'].nunique().reset_index()
    print(temp['grant_id'].value_counts(normalize=True).head(3))
    print("(Total researchers: {})".format(temp.shape[0]))

### Grant Characteristics

In [ ]:
results['grant_characteristics'] = {}

df = pd.merge(funded_grnts_funder, funded_grnts_info, how='left', on='grant_id')
print("Average Grant Amount: {}".format(df['funding_amount'].mean()))
print("Average Funding Length: {}".format(df['funding_len'].mean()))
print("Average Team Size: {}".format(df['nb_rsrs'].mean()))
results['grant_characteristics']['overall'] = df[['funding_amount', 'funding_len', 'nb_rsrs']].describe()

funded_grnts_amt_avg = []
funded_grnts_amt_med = []
funded_grnts_len_avg = []
funded_grnts_len_med = []
funded_grnts_team_size_avg = []
funded_grnts_team_size_med = []
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder].copy()
    print(temp[['funding_amount', 'funding_len', 'nb_rsrs']].describe())
    print("(Total grants: {})".format(temp.shape[0]))
    results['grant_characteristics'][funder] = temp[['funding_amount', 'funding_len', 'nb_rsrs']].describe()

In [ ]:
temp = pd.merge(funded_grnts_funder, funded_grnts_info, how='left', on='grant_id')
temp['year'] = pd.DatetimeIndex(temp['start_date']).year
pd.crosstab(temp['funder_name'], temp['year'])

## 2. Prior and Subsequent Grants

In [ ]:
grnts_stats = grant_groupby(grnts_mrg)

In [ ]:
results['pre_grnt_stats'] = {}
results['post_grnt_stats'] = {}

cols = ['nb_grnts', 'fund_amt', 'avg_fund_len', 'avg_team_size']

temp = grnts_stats[grnts_stats['status']=="pre"].copy()
print("Average Funding Length Pre-Grant: {}".format(temp['avg_fund_len'].mean()))
print("Average Team Size Pre-Grant: {}".format(temp['avg_team_size'].mean()))
print("Average Funding Amount Pre-Grant: {}".format(temp['fund_amt'].mean()))
print("Average Total Number of Grants Pre-Grant: {}".format(temp['nb_grnts'].mean()))
results['pre_grnt_stats']['overall'] = temp[cols].describe()

temp = grnts_stats[grnts_stats['status']=="post"].copy()
print("\nAverage Funding Length Post-Grant: {}".format(temp['avg_fund_len'].mean()))
print("Average Team Size Post-Grant: {}".format(temp['avg_team_size'].mean()))
print("Average Funding Amount Post-Grant: {}".format(temp['fund_amt'].mean()))
print("Average Total Number of Grants Post-Grant: {}".format(temp['nb_grnts'].mean()))
results['post_grnt_stats']['overall'] = temp[cols].describe()

print("\n")

print("Pre-Funding Grant Statistics:")
df = grnts_stats[grnts_stats['status']=="pre"].copy()
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp[cols].describe())
    results['pre_grnt_stats'][funder] = temp[cols].describe()

print("\n")

print("Post-Funding Grant Statistics:")
df = grnts_stats[grnts_stats['status']=="post"].copy()
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp[cols].describe())
    results['pre_grnt_stats'][funder] = temp[cols].describe()

## 3. Prior and Subsequent Publications

In [ ]:
pubs_stats = pub_groupby(pubs_mrg)

In [ ]:
results['pre_pubs_stats'] = {}
results['post_pubs_stats'] = {}

cols = ['citations_per_pub', 'nb_pubs', 'team_size', 'nb_collabs', 'nb_collab_countries']

temp = pubs_stats[pubs_stats['status']=="pre"].copy()
print("Average Citations Pre-Grant: {}".format(temp['citations_per_pub'].mean()))
print("Average Total Number of Publications Pre-Grant: {}".format(temp['nb_pubs'].mean()))
results['pre_pubs_stats']['overall'] = temp[cols].describe()

temp = pubs_stats[pubs_stats['status']=="post"].copy()
print("\nAverage Citations Post-Grant: {}".format(temp['citations_per_pub'].mean()))
print("Average Total Number of Publications Post-Grant: {}".format(temp['nb_pubs'].mean()))
results['post_pubs_stats']['overall'] = temp[cols].describe()

print("\n")

print("Pre-Funding Publication Statistics:")
df = pubs_stats[pubs_stats['status']=="pre"].copy()
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp[cols].describe())
    results['pre_pubs_stats'][funder] = temp[cols].describe()

print("\n")

print("Post-Funding Publication Statistics:")
df = pubs_stats[pubs_stats['status']=="post"].copy()
cols = ['citations_per_pub', 'nb_pubs', 'team_size', 'nb_collabs', 'nb_collab_countries']
for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp[cols].describe())
    results['post_pubs_stats'][funder] = temp[cols].describe()

## 4. Topic Analyses

### Funded Grant RCDC Codes Analysis

In [ ]:
df = pd.merge(funded_grnts_info, grnts_rcdc, how='left', on='grant_id')

In [ ]:
print("Most Common RCDC Codes:")
print(df['rcdc_name'].value_counts().head())
funded_grnts_rcdc_tot = df['rcdc_name'].value_counts().index[0:5]

In [ ]:
df = pd.merge(funded_grnts_funder, grnts_rcdc, how='left', on='grant_id')

In [ ]:
print("Average number of RCDC's per Grant:\n")
print(df[['funder_name', 'grant_id', 'nb_rcdc']].drop_duplicates().groupby('funder_name')['nb_rcdc'].mean())

In [ ]:
funded_grnts_rcdc_1 = []
funded_grnts_rcdc_2 = []
funded_grnts_rcdc_3 = []
funded_grnts_rcdc_4 = []
funded_grnts_rcdc_5 = []

for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print(temp['rcdc_name'].value_counts().head())
    funded_grnts_rcdc_1.append(temp['rcdc_name'].value_counts().index[0])
    funded_grnts_rcdc_2.append(temp['rcdc_name'].value_counts().index[1])
    funded_grnts_rcdc_3.append(temp['rcdc_name'].value_counts().index[2])
    funded_grnts_rcdc_4.append(temp['rcdc_name'].value_counts().index[3])
    funded_grnts_rcdc_5.append(temp['rcdc_name'].value_counts().index[4])

In [ ]:
pubs_cso_1 = pubs_cso.drop_duplicates('pub_id', keep='first').reset_index(drop=True)

### Publication CSO Codes

In [ ]:
df = pd.merge(pubs_mrg, pubs_cso, how='left', on='pub_id')
df = df[['funder_name', 'pub_id', 'cso', 'nb_cso']].drop_duplicates().reset_index(drop=True)

In [ ]:
print("Average number of CSO Codes per Publication Associated to Grant:\n")
print(df[['funder_name', 'pub_id', 'nb_cso']].drop_duplicates().groupby('funder_name')['nb_cso'].mean())

In [ ]:
cso_1 = []
cso_2 = []
cso_3 = []
cso_4 = []
cso_5 = []

for funder in funders:
    print("\n{}:".format(funder))
    temp = df[df['funder_name']==funder]
    print("Number of Publications with missing CSO Codes: {}".format(temp[temp['cso'].isnull()].shape[0]))
    print(temp['cso'].value_counts().head())
    cso_1.append(temp['cso'].value_counts().index[0])
    cso_2.append(temp['cso'].value_counts().index[1])
    cso_3.append(temp['cso'].value_counts().index[2])
    cso_4.append(temp['cso'].value_counts().index[3])
    cso_5.append(temp['cso'].value_counts().index[4])

### CSO * RCDC Distributions

In [107]:
rcdc_codes_to_remove = ['CANCER', 'CLINICAL RESEARCH', 'PREVENTION', 'DIAGNOSTIC RADIOLOGY', 
                        'BEHAVIORAL AND SOCIAL SCIENCE', 'PATIENT SAFETY', 'HEALTH SERVICES', 'IMMUNIZATION', 
                        'COMPLEMENTARY AND ALTERNATIVE MEDICINE', 'BASIC BEHAVIORAL AND SOCIAL SCIENCE', 
                        'COMPARATIVE EFFECTIVENESS RESEARCH', 'CLINICAL TRIALS AND SUPPORTIVE ACTIVITIES',
                        'NETWORKING AND INFORMATION TECHNOLOGY R&D', 'BURDEN OF ILLNESS']

In [108]:
topic_distributions = pd.merge(pubs_mrg, pubs_cso, on='pub_id', how='left')
topic_distributions = pd.merge(topic_distributions, cso_lookup, on='cso_name', how='left')
topic_distributions = pd.merge(topic_distributions, pubs_rcdc, on='pub_id', how='left')

In [109]:
# Which CSO?
topic_distributions.rename(columns={'cso_cat': 'cso'}, inplace=True)

In [110]:
# Restrictions
topic_distributions = topic_distributions[topic_distributions['status']=="post"]

In [112]:
# RCDC:
df = topic_distributions[['funder_name', 'rsr_id', 'pub_id', 'rcdc_name']].drop_duplicates()
df = df[df['rcdc_name'].notnull()].reset_index(drop=True)
df = df[df['rcdc_name']!="NAN"].reset_index(drop=True)    
for rcdc in rcdc_codes_to_remove:
    df = df[df['rcdc_name']!=rcdc].reset_index(drop=True)
# df = df[df['rcdc_name'].str.upper().str.contains("CANCER")]

out = pd.DataFrame()
for agency in list(df['funder_name'].unique()):
    temp = df[df['funder_name']==agency].copy().reset_index(drop=True)
    den = temp[['pub_id', 'rsr_id']].drop_duplicates().shape[0]
    temp['nb_rcdc'] = temp.groupby(['pub_id', 'rsr_id'])['pub_id'].transform('count')
    temp[agency+' pubs'] = 1/temp['nb_rcdc']
    temp = temp.groupby('rcdc_name')[agency+' pubs'].sum().reset_index()
    print(temp[agency+' pubs'].sum())
    temp[agency+' share'] = temp[agency+' pubs']/den
    print(temp[agency+' share'].sum())
    temp.sort_values(agency+' pubs', ascending=False, inplace=True)
    del temp[agency+' pubs']
    
    if out.empty:
        out = temp
    else:
        out = pd.merge(out, temp, how='outer', on='rcdc_name')
    
out['average_share'] = (out['INCa/INSERM/DGOS share']+out['National Cancer Institute share']
                          +out['Cancer Research UK share']+out['Wellcome Trust share']
                          +out['National Health and Medical Research Council share'])/5
    
out.to_csv('../output/top_topics/rcdc.csv'.format(agency.replace("/", "-")), index=False)
print(out.shape[0])

25809.00000000001
1.0000000000000004
234298.00000000748
1.0000000000000318
23890.000000000036
1.0000000000000013
6151.000000000011
1.0000000000000018
49135.99999999984
0.9999999999999968
240


In [114]:
# CSO
df = topic_distributions[['funder_name', 'rsr_id', 'pub_id', 'cso']].drop_duplicates()
df = df[df['cso'].notnull()].reset_index(drop=True)
df = df[df['cso']!="NAN"].reset_index(drop=True)    

out = pd.DataFrame()
for agency in list(df['funder_name'].unique()):
    temp = df[df['funder_name']==agency].copy().reset_index(drop=True)
    den = temp[['pub_id', 'rsr_id']].drop_duplicates().shape[0]
    temp['nb_cso'] = temp.groupby(['pub_id', 'rsr_id'])['pub_id'].transform('count')
    temp[agency+' pubs'] = 1/temp['nb_cso']
    temp = temp.groupby('cso')[agency+' pubs'].sum().reset_index()
    print(temp[agency+' pubs'].sum())
    temp[agency+' share'] = temp[agency+' pubs']/den
    print(temp[agency+' share'].sum())
    temp.sort_values(agency+' pubs', ascending=False, inplace=True)
    del temp[agency+' pubs']
    
    if out.empty:
        out = temp
    else:
        out = pd.merge(out, temp, how='outer', on='cso')

out['average_share'] = (out['INCa/INSERM/DGOS share']+out['National Cancer Institute share']
                          +out['Cancer Research UK share']+out['Wellcome Trust share']
                          +out['National Health and Medical Research Council share'])/5
    
out.to_csv('../output/top_topics/cso.csv'.format(agency.replace("/", "-")), index=False)
print(out.shape[0])

21192.0
1.0
202453.99999999985
0.9999999999999993
22672.0
1.0
5159.0
1.0
41108.999999999985
0.9999999999999997
6


In [139]:
df = topic_distributions[['funder_name', 'rsr_id', 'pub_id', 'rcdc_name', 'cso']].drop_duplicates()
df = df[df['rcdc_name'].notnull()].reset_index(drop=True)
df = df[df['rcdc_name']!="NAN"].reset_index(drop=True)    
for rcdc in rcdc_codes_to_remove:
    df = df[df['rcdc_name']!=rcdc].reset_index(drop=True)
# df = df[df['rcdc_name'].str.upper().str.contains("CANCER")]
df = df[df['cso'].notnull()].reset_index(drop=True)
df = df[df['cso']!="NAN"].reset_index(drop=True)    
df['rcdc_cso'] = df['rcdc_name']+" * "+df['cso']

out = pd.DataFrame()
for agency in list(df['funder_name'].unique()):
    temp = df[df['funder_name']==agency].copy().reset_index(drop=True)
    den = temp[['pub_id', 'rsr_id']].drop_duplicates().shape[0]
    print(den)
    temp['nb'] = temp.groupby(['pub_id', 'rsr_id'])['pub_id'].transform('count')
    temp[agency+' pubs'] = 1/temp['nb']
    temp = temp.groupby('rcdc_cso')[agency+' pubs'].sum().reset_index()
    print(temp[agency+' pubs'].sum())
#     temp[agency+' share'] = temp[agency+' pubs']/den
#     print(temp[agency+' share'].sum())
    temp.sort_values(agency+' pubs', ascending=False, inplace=True)
#     del temp[agency+' share']
    
    if out.empty:
        out = temp
    else:
        out = pd.merge(out, temp, how='outer', on='rcdc_cso')

        
# out['average_share'] = (out['INCa/INSERM/DGOS share']+out['National Cancer Institute share']
#                           +out['Cancer Research UK share']+out['Wellcome Trust share']
#                           +out['National Health and Medical Research Council share'])/5
    
out.to_csv('../output/top_topics/rcdc_cso.csv'.format(agency.replace("/", "-")), index=False)
print(out.shape[0])

17984
17984.000000000004
178308
178308.00000000006
18547
18546.999999999993
4416
4416.0
34616
34615.999999999985


KeyError: 'INCa/INSERM/DGOS share'

In [121]:
df = topic_distributions[['funder_name', 'rsr_id', 'pub_id', 'year', 'rcdc_name', 'cso']].drop_duplicates()
df = df[df['rcdc_name'].notnull()].reset_index(drop=True)
df = df[df['rcdc_name']!="NAN"].reset_index(drop=True)    
for rcdc in rcdc_codes_to_remove:
    df = df[df['rcdc_name']!=rcdc].reset_index(drop=True)
# df = df[df['rcdc_name'].str.upper().str.contains("CANCER")]
df = df[df['cso'].notnull()].reset_index(drop=True)
df = df[df['cso']!="NAN"].reset_index(drop=True)    
df['rcdc_cso'] = df['rcdc_name']+" * "+df['cso']

out = pd.DataFrame()
for agency in list(df['funder_name'].unique()):
    temp = df[df['funder_name']==agency].copy().reset_index(drop=True)
    den = temp[['pub_id', 'rsr_id']].drop_duplicates().shape[0]
    print(den)
    temp['nb'] = temp.groupby(['pub_id', 'rsr_id'])['pub_id'].transform('count')
    temp[agency+' pubs'] = 1/temp['nb']
    temp = temp.groupby(['rcdc_cso', 'year'])[agency+' pubs'].sum().reset_index()
    print(temp[agency+' pubs'].sum())
#     temp[agency+' share'] = temp[agency+' pubs']
#     print(temp[agency+' share'].sum())
    temp.sort_values(agency+' pubs', ascending=False, inplace=True)
    
    if out.empty:
        out = temp
    else:
        out = pd.merge(out, temp, how='outer', on=['rcdc_cso', 'year'])

        
# out['average_share'] = (out['INCa/INSERM/DGOS share']+out['National Cancer Institute share']
#                           +out['Cancer Research UK share']+out['Wellcome Trust share']
#                           +out['National Health and Medical Research Council share'])/5
    
out.to_csv('../output/top_topics/rcdc_cso_by_year.csv'.format(agency.replace("/", "-")), index=False)
print(out.shape[0])

17984
17984.0
178308
178308.0
18547
18547.0
4416
4416.0
34616
34616.0
8999


## 5. Researcher Level Statistics

In [ ]:
rsrs.describe(include='all')

## 6. Balanced Prior Publications

For this section, we restrict data to the first grant of every researcher (which is why averages are not quite the same as before).

In [85]:
pubs_stats = pub_groupby(pubs_mrg, yearly=True)

In [86]:
balanced_pubs_stats = pd.merge(pubs_stats, propensity_scores, on=['rsr_id', 'funder_name', 'grant_id'])
balanced_pubs_stats = pd.merge(balanced_pubs_stats, rsrs[['rsr_id', 'rsr_gender', 'rsr_career_start_year']]
                               , how='left', on='rsr_id')
balanced_pubs_stats['rsr_career_age'] = balanced_pubs_stats['year']-balanced_pubs_stats['rsr_career_start_year']
balanced_pubs_stats = balanced_pubs_stats[balanced_pubs_stats['weight'].notnull()]

In [103]:
cols = ['citations_per_pub', 'nb_pubs', 'team_size', 'nb_collabs', 'nb_collab_countries', 'rsr_career_age']

In [106]:
df = pd.DataFrame()
for var in cols:
    temp = balanced_pubs_stats[['funder_name', var, 'status']].copy()
    temp = pd.pivot_table(temp, index='status', columns='funder_name', values=var, aggfunc='mean').reset_index()
    temp['var'] = var
    temp['propensity_weight'] = False
    if df.empty:
        df = temp
    else:
        df = pd.concat([df, temp], sort=False)
df[['propensity_weight', 'status', 'var']+funders].sort_values(['status', 'var'], ascending=[False,True])    

funder_name,propensity_weight,status,var,Cancer Research UK,INCa/INSERM/DGOS,National Cancer Institute,National Health and Medical Research Council,Wellcome Trust
1,False,pre,citations_per_pub,68.283640,44.738510,65.441251,57.661015,61.658512
1,False,pre,nb_collab_countries,1.256419,1.936290,1.343677,1.557676,1.402547
1,False,pre,nb_collabs,13.233238,29.136256,20.440187,17.213628,14.612403
1,False,pre,nb_pubs,2.958393,5.682977,4.812341,3.714716,3.299003
1,False,pre,rsr_career_age,7.039230,11.917941,11.120431,8.861125,7.124585
1,False,pre,team_size,6.892701,8.868371,6.972869,8.233057,7.480734
0,False,post,citations_per_pub,40.038041,25.674743,34.756748,30.560683,38.049873
0,False,post,nb_collab_countries,2.217404,3.317839,2.213013,2.738322,2.870432
0,False,post,nb_collabs,27.269187,52.785729,38.925458,35.345756,36.549502
0,False,post,nb_pubs,4.216690,8.198165,6.804695,5.806128,4.813289


In [108]:
df = pd.DataFrame()
for var in cols:
    temp = balanced_pubs_stats[['funder_name', var, 'status', 'weight']].copy()
    temp[var+'_temp'] = temp[var]*temp['weight']
    temp = pd.pivot_table(temp, index='status', columns='funder_name', 
                          values=var+'_temp', aggfunc='sum').reset_index()
    temp['var'] = var
    
    for funder in funders:
        pre_den = (np.where(balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder]['status']=="post", 0, 
                            np.where(balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder][var].notnull(), 
                                     balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder]['weight'], 
                                     0))).sum()
        post_den = (np.where(balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder]['status']=="pre", 0, 
                            np.where(balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder][var].notnull(), 
                                     balanced_pubs_stats[balanced_pubs_stats['funder_name']==funder]['weight'], 
                                     0))).sum()
        temp[funder] = np.where(temp['status']=="pre", temp[funder]/pre_den, temp[funder]/post_den)
            
    temp['var'] = var
    temp['propensity_weight'] = True
    if df.empty:
        df = temp
    else:
        df = pd.concat([df, temp])
df[['propensity_weight', 'status', 'var']+funders].sort_values(['status', 'var'], ascending=[False,True])

funder_name,propensity_weight,status,var,Cancer Research UK,INCa/INSERM/DGOS,National Cancer Institute,National Health and Medical Research Council,Wellcome Trust
1,True,pre,citations_per_pub,64.344293,44.738510,62.380557,56.050739,41.479237
1,True,pre,nb_collab_countries,2.020728,1.936290,1.486949,2.000478,1.651419
1,True,pre,nb_collabs,22.657661,29.136256,23.429964,21.552185,16.065760
1,True,pre,nb_pubs,4.692293,5.682977,5.244051,4.704514,3.850436
1,True,pre,rsr_career_age,12.373125,11.917941,12.941043,11.516370,12.027242
1,True,pre,team_size,7.279211,8.868371,7.153443,7.382119,6.499943
0,True,post,citations_per_pub,36.575277,25.674743,34.595996,29.514192,32.887755
0,True,post,nb_collab_countries,3.607258,3.317839,2.493580,3.342572,2.779446
0,True,post,nb_collabs,49.617983,52.785729,45.176102,44.449205,32.532506
0,True,post,nb_pubs,6.578703,8.198165,7.456653,7.218277,4.886758


In [46]:
df = pd.DataFrame()
temp = balanced_pubs_stats[['funder_name', 'rsr_gender', 'status']]
temp = pd.crosstab(temp['funder_name'], temp['rsr_gender'],normalize='index').reset_index()
if df.empty:
    df = temp
else:
    df = pd.concat([df, temp])
df[['funder_name', 'F', 'M', 'UNKNOWN']]

rsr_gender,funder_name,F,M,UNKNOWN
0,Cancer Research UK,0.286020,0.631241,0.082739
1,INCa/INSERM/DGOS,0.309888,0.594292,0.095821
2,National Cancer Institute,0.282131,0.529103,0.188766
3,National Health and Medical Research Council,0.327976,0.573581,0.098443
4,Wellcome Trust,0.318937,0.578073,0.102990


In [47]:
df = pd.DataFrame()
temp = balanced_pubs_stats[['funder_name', 'rsr_gender', 'status', 'weight']]
temp = pd.crosstab(temp['funder_name'], temp['rsr_gender'], temp['weight'], aggfunc = sum, 
                   normalize='index').reset_index()
if df.empty:
    df = temp
else:
    df = pd.concat([df, temp])
df[['funder_name', 'F', 'M', 'UNKNOWN']]

rsr_gender,funder_name,F,M,UNKNOWN
0,Cancer Research UK,0.348805,0.586650,0.064545
1,INCa/INSERM/DGOS,0.309888,0.594292,0.095821
2,National Cancer Institute,0.364846,0.497372,0.137781
3,National Health and Medical Research Council,0.371612,0.526615,0.101773
4,Wellcome Trust,0.495249,0.479236,0.025515


## Creating Output Comparison Table

### Overall Table

In [ ]:
# df = pd.DataFrame({
#     'funded_grnts_per_rsrs_tot': funded_grnts_per_rsrs_tot
#     , 'agencies_per_rsrs_tot': agencies_per_rsrs_tot
#     , 'funded_amt_tot': funded_amt_tot
#     , 'funded_len_tot': funded_len_tot
#     , 'nb_grnt_rsrs_tot': nb_grnt_rsrs_tot
#     , 'grnt_fund_len_tot': grnt_fund_len_tot
#     , 'grnt_team_size_tot': grnt_team_size_tot
#     , 'grnt_fund_amt_tot': grnt_fund_amt_tot
#     , 'pre_avg_fund_len_tot': pre_avg_fund_len_tot
#     , 'pre_avg_team_size_tot': pre_avg_team_size_tot
#     , 'pre_fund_amt_tot': pre_fund_amt_tot
#     , 'pre_nb_grnts_tot': pre_nb_grnts_tot
#     , 'post_avg_fund_len_tot': post_avg_fund_len_tot
#     , 'post_avg_team_size_tot': post_avg_team_size_tot
#     , 'post_fund_amt_tot': post_fund_amt_tot
#     , 'post_nb_grnts_tot': post_nb_grnts_tot
#     , 'pub_cit_tot': pub_cit_tot
#     , 'pub_team_size_tot': pub_team_size_tot
#     , 'pre_pub_cit_tot': pre_pub_cit_tot
#     , 'pre_nb_pubs_tot': pre_nb_pubs_tot
#     , 'post_pub_cit_tot': post_pub_cit_tot
#     , 'post_nb_pubs_tot': post_nb_pubs_tot
#     }, index=['mean', 'std']).transpose()
# df

In [ ]:
# # Export to Excel
# ls = !ls ../output/
# if 'comparison_statistics.xlsx' in ls:
#     book = load_workbook('../output/comparison_statistics.xlsx')
#     writer = pd.ExcelWriter('../output/comparison_statistics.xlsx', engine='openpyxl') 
#     writer.book = book
#     writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
#     df.to_excel(writer, "raw_all")
#     writer.save()
# else:
#     df.to_excel('../output/comparison_statistics.xlsx', sheet_name = 'raw_all')

### Table by Funding Agency

In [ ]:
# df = pd.DataFrame({'rcdc_1':rcdc_1
#                     , 'rcdc_2':rcdc_2
#                     , 'rcdc_3':rcdc_3
#                     , 'rcdc_4':rcdc_4
#                     , 'rcdc_5':rcdc_5                   
#                     , 'nb_unique_rsrs': nb_unique_rsrs
#                     , 'nb_unique_grnts': nb_unique_grnts
#                     , 'funded_amt_avg': funded_amt_avg
#                     , 'funded_amt_med': funded_amt_med
#                     , 'funded_len_avg': funded_len_avg
#                     , 'funded_len_med': funded_len_med
#                     , 'nb_grnt_rsrs_avg': nb_grnt_rsrs_avg
#                     , 'nb_grnt_rsrs_med': nb_grnt_rsrs_med
#                     , 'pre_avg_fund_len_avg': pre_avg_fund_len_avg
#                     , 'pre_avg_fund_len_med': pre_avg_fund_len_med
#                     , 'pre_avg_team_size_avg': pre_avg_team_size_avg
#                     , 'pre_avg_team_size_med': pre_avg_team_size_med
#                     , 'pre_fund_amt_avg': pre_fund_amt_avg
#                     , 'pre_fund_amt_med': pre_fund_amt_med
#                     , 'pre_nb_grnts_avg': pre_nb_grnts_avg
#                     , 'pre_nb_grnts_med': pre_nb_grnts_med
#                     , 'post_avg_fund_len_avg': post_avg_fund_len_avg
#                     , 'post_avg_fund_len_med': post_avg_fund_len_med
#                     , 'post_avg_team_size_avg': post_avg_team_size_avg
#                     , 'post_avg_team_size_med': post_avg_team_size_med
#                     , 'post_fund_amt_avg': post_fund_amt_avg
#                     , 'post_fund_amt_med': post_fund_amt_med
#                     , 'post_nb_grnts_avg': post_nb_grnts_avg
#                     , 'post_nb_grnts_med': post_nb_grnts_med
#                     , 'pre_citations_avg': pre_citations_avg
#                     , 'pre_nb_pubs_avg': pre_nb_pubs_avg
#                     , 'pre_citations_med': pre_citations_med
#                     , 'pre_nb_pubs_med': pre_nb_pubs_med
#                     , 'post_citations_avg': post_citations_avg
#                     , 'post_citations_med': post_citations_med
#                     , 'post_nb_pubs_avg': post_nb_pubs_avg
#                     , 'post_nb_pubs_med': post_nb_pubs_med
#                    , 'pre_team_size_avg': pre_team_size_avg
#                    , 'pre_team_size_med': pre_team_size_med
#                    , 'post_team_size_avg': post_team_size_avg
#                    , 'post_team_size_med': post_team_size_med
#                   }, index=funders).transpose()
# df

In [ ]:
# # Export to Excel
# ls = !ls ../output/
# if 'comparison_statistics.xlsx' in ls:
#     book = load_workbook('../output/comparison_statistics.xlsx')
#     writer = pd.ExcelWriter('../output/comparison_statistics.xlsx', engine='openpyxl') 
#     writer.book = book
#     writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
#     df.to_excel(writer, "raw_by_agency")
#     writer.save()
# else:
#     df.to_excel('../output/comparison_statistics.xlsx', sheet_name = 'raw_by_agency')

### Balance Table

## Preparing Data for Regression Analysis

In [35]:
yearly = True

### Researcher Info

In [36]:
rsrs.describe(include='all')

,rsr_id,first_name,last_name,rsr_gender,rsr_career_start_year,rsr_affiliation,rsr_affiliation_id,rsr_country,rsr_city,orcid_confirmed,rsr_nb_early_pubs,rsr_nb_early_citations,cso_0,cso_1,cso_10,cso_11,cso_12,cso_13,cso_14,cso_15,cso_16,cso_17,cso_18,cso_19,cso_2,cso_20,cso_21,cso_22,cso_23,cso_24,cso_25,cso_26,cso_27,cso_28,cso_29,cso_3,cso_30,cso_31,cso_32,cso_33,cso_4,cso_5,cso_6,cso_7,cso_8,cso_9,rcdc_0,rcdc_1,rcdc_10,rcdc_100,...,rcdc_54,rcdc_55,rcdc_56,rcdc_57,rcdc_58,rcdc_59,rcdc_6,rcdc_60,rcdc_61,rcdc_62,rcdc_63,rcdc_64,rcdc_65,rcdc_66,rcdc_67,rcdc_68,rcdc_69,rcdc_7,rcdc_70,rcdc_71,rcdc_72,rcdc_73,rcdc_74,rcdc_75,rcdc_76,rcdc_77,rcdc_78,rcdc_79,rcdc_8,rcdc_80,rcdc_81,rcdc_82,rcdc_83,rcdc_84,rcdc_85,rcdc_86,rcdc_87,rcdc_88,rcdc_89,rcdc_9,rcdc_90,rcdc_91,rcdc_92,rcdc_93,rcdc_94,rcdc_95,rcdc_96,rcdc_97,rcdc_98,rcdc_99
count,14004,13978,14004,14004,14004.000000,12173,9804,9804,9804,14004,11897.000000,11897.000000,63.000000,754.000000,22.000000,621.000000,4.0,1150.000000,240.000000,314.000000,130.000000,224.000000,2703.000000,259.000000,1089.000000,596.000000,175.000000,67.000000,63.000000,34.000000,13.000000,34.000000,220.000000,549.000000,1608.000000,627.000000,796.000000,335.000000,399.000000,81.000000,99.000000,75.000000,3.000000,7.0,224.00000,137.000000,55.000000,5.000000,2.0,4.00,...,18.000000,47.000000,21.000000,26.000000,52.000000,185.000000,3.0,126.000000,189.000000,557.000000,1450.000000,9.000000,24.000000,5.000000,141.000000,7.0,1.0,37.000000,17.000000,9.000000,446.000000,8.000000,9.000000,18.000000,208.000000,134.000000,2.0,2.000000,37.000000,5.0,3.0,2.0,35.000000,5.0,146.000000,163.000000,3731.000000,5.0,1.0,18.000000,8.0,332.000000,329.000000,121.000000,1231.000000,107.000000,50.000000,30.000000,458.000000,14.000000
unique,14004,7683,9132,3,NaN,4030,1709,60,727,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,ur.0646414361.37,David,WANG,M,NaN,Harvard University,grid.38142.3c,US,Cambridge,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,142,91,7720,NaN,197,197,6492,414,13834,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,1996.344045,NaN,NaN,NaN,NaN,NaN,3.131041,210.915777,1.190476,1.472149,1.409091,1.399356,1.5,1.760870,1.595833,1.761146,1.284615,1.616071,1.669256,1.494208,1.586777,1.380872,1.160000,1.179104,1.349206,1.088235,1.153846,1.235294,1.368182,1.683060,1.640547,1.524721,1.662060,1.567164,1.513784,1.419753,1.363636,1.173333,1.333333,1.0,1.40625,1.306569,1.290909,1.200000,1.0,1.25,...,1.611111,1.468085,1.285714,1.615385,1.269231,1.335135,1.0,1.325397,1.550265,1.834829,1.647586,1.111111,2.041667,1.200000,1.496454,1.0,1.0,1.270270,1.235294,1.333333,1.392377,1.125000,1.222222,1.277778,1.495192,1.313433,1.0,1.500000,1.297297,1.0,1.0,1.0,1.085714,2.0,1.452055,1.245399,1.962745,1.0,1.0,1.277778,1.0,1.454819,1.407295,1.487603,1.706742,1.794393,1.700000,1.666667,1.539301,1.071429
std,NaN,NaN,NaN,NaN,10.437329,NaN,NaN,NaN,NaN,NaN,2.444039,477.321106,0.715207,0.853020,0.503236,0.891682,1.0,1.374022,1.263911,1.346022,0.625487,1.364150,1.065632,0.90

### Grant Information

In [37]:
print(funded_grnts_info.set_index(['grant_id']).index.is_unique)
print(funded_grnts_info.shape)
funded_grnts_info.head()

True
(16925, 7)


,grant_id,start_date,end_date,funding_amount,funding_len,start_year,nb_rsrs
0,grant.7426242,2011-12-13,2013-06-12,65061.0,1.498630,2011,1
1,grant.7426178,2011-11-01,2014-11-01,240110.0,3.002740,2011,2
2,grant.7154464,2010-06-01,2013-06-01,248109.0,3.002740,2010,2
3,grant.7154483,2010-11-04,2013-11-04,400306.0,3.002740,2010,1
4,grant.7154359,2007-12-18,2010-06-30,140617.0,2.534247,2007,2


### Prior/Subsequent Grant Statistics

In [38]:
grnts_stats = grant_groupby(grnts_mrg, yearly=yearly)

In [39]:
# Check if unit of observation is Funder-Grant-Researcher:
cols = ['funder_name', 'grant_id', 'rsr_id', 'status']
if yearly == True:
    cols+=['year']
print(grnts_stats.set_index(cols).index.is_unique)
print(grnts_stats.shape)
grnts_stats.head()

True
(242011, 9)


,funder_name,grant_id,rsr_id,status,year,nb_grnts,fund_amt,avg_fund_len,avg_team_size
0,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2012,0,NaN,NaN,NaN
1,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2013,0,NaN,NaN,NaN
2,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2014,0,NaN,NaN,NaN
3,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2015,0,NaN,NaN,NaN
4,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2016,0,NaN,NaN,NaN


### Prior/Subsequent Publication Statistics

In [44]:
pubs_stats = pub_groupby(pubs_mrg, yearly=yearly)

In [45]:
# Check if unit of observation is Funder-Grant-Researcher:
cols = ['funder_name', 'grant_id', 'rsr_id', 'status']
if yearly == True:
    cols+=['year']
print(pubs_stats.set_index(cols).index.is_unique)
print(pubs_stats.shape)
pubs_stats.head()

True
(242011, 12)


,funder_name,grant_id,rsr_id,status,year,nb_pubs,citations_per_pub,team_size,nb_collabs,nb_collab_countries,nb_pubs_in_topic,pct_pubs_in_topic
0,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2012,5,9.600000,11.000000,42,2,0.0,0.0
1,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2013,9,13.333333,9.444444,58,6,0.0,0.0
2,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2014,7,13.285714,12.714286,53,7,0.0,0.0
3,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2015,8,4.750000,8.250000,47,3,0.0,0.0
4,INCa/INSERM/DGOS,grant.7426242,ur.0642054564.81,post,2016,7,2.142857,9.857143,44,5,0.0,0.0


### Add CSO Codes

In [61]:
pubs_cso.head()

,pub_id,cso_name,nb_cso
0,pub.1000000285,Cancer Initiation: Alterations in Chromosomes,2
1,pub.1000000285,Normal Functioning,2
2,pub.1000000357,Normal Functioning,1
3,pub.1000000436,Exogenous Factors in the Origin and Cause of C...,1
4,pub.1000000461,Normal Functioning,1


In [63]:
pubs_cso_features = pd.merge(pubs_mrg[['funder_name', 'grant_id', 'rsr_id', 'year', 'pub_id']], pubs_cso, 
                             how='inner', on='pub_id')
pubs_cso_features['cso_id'] = pubs_cso_features.groupby(['cso_name']).ngroup()
pubs_cso_features['cso'] = "cso_"+pubs_cso_features['cso_id'].astype(str)

In [64]:
pubs_cso_features = pubs_cso_features.groupby(['funder_name', 'grant_id', 'rsr_id', 'cso'])
pubs_cso_features = pd.DataFrame({'nb_pubs': pubs_cso_features['pub_id'].count()}).reset_index()

In [65]:
pubs_cso_features = pd.pivot_table(pubs_cso_features, index=['funder_name', 'grant_id', 'rsr_id'], 
                                   columns='cso', values='nb_pubs', aggfunc='sum').reset_index()

In [66]:
print(pubs_cso_features.set_index(['funder_name', 'grant_id', 'rsr_id']).index.is_unique)
print(pubs_cso_features.shape)
pubs_cso_features.head()

True
(18903, 37)


cso,funder_name,grant_id,rsr_id,cso_0,cso_1,cso_10,cso_11,cso_12,cso_13,cso_14,cso_15,cso_16,cso_17,cso_18,cso_19,cso_2,cso_20,cso_21,cso_22,cso_23,cso_24,cso_25,cso_26,cso_27,cso_28,cso_29,cso_3,cso_30,cso_31,cso_32,cso_33,cso_4,cso_5,cso_6,cso_7,cso_8,cso_9
0,Cancer Research UK,grant.5133968,ur.0676015717.39,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.0,1.0,3.0,NaN,NaN,2.0,NaN,NaN,1.0,NaN
1,Cancer Research UK,grant.5133977,ur.01247755277.14,NaN,15.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cancer Research UK,grant.5133988,ur.01236232563.71,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cancer Research UK,grant.5133993,ur.01342746606.50,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,14.0,NaN,54.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,13.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cancer Research UK,grant.5133993,ur.0575573745.40,NaN,6.0,NaN,28.0,NaN,1.0,47.0,NaN,NaN,NaN,5.0,NaN,NaN,1.0,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,1.0,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Combine All

In [47]:
df = pd.merge(funded_grnts[['funder_name', 'grant_id', 'start_year', 'start_date', 'rsr_id']].drop_duplicates(), 
              merge_key, how='left', on='start_year')
df = pd.merge(funded_grnts_funder, funded_grnts_rsr, how='outer', on='grant_id')
df = pd.merge(df, funded_grnts_info, how='outer', on='grant_id')
df = pd.merge(df, merge_key, how='left', on='start_year')
df = pd.merge(df, grnts_stats, how='outer', on=['funder_name', 'grant_id', 'rsr_id', 'year', 'status'])
df = pd.merge(df, pubs_stats, how='outer', on=['funder_name', 'grant_id', 'rsr_id', 'year', 'status'])
df = pd.merge(df, rsrs, how='left', on='rsr_id')
df = pd.merge(df, propensity_scores[['rsr_id', 'inca_prob', 'weight']], on='rsr_id', how='left')
df['rsr_career_age'] = df['year']-df['rsr_career_start_year']
# df = pd.merge(df, pubs_cso_features, how='outer', on=['funder_name', 'grant_id', 'rsr_id'])

In [48]:
print(df.set_index(['funder_name', 'grant_id', 'rsr_id', 'year', 'status']).index.is_unique)
print(df.shape)
df.describe(include='all')

True
(242011, 305)


,funder_name,grant_id,rsr_id,start_date,end_date,funding_amount,funding_len,start_year,nb_rsrs,status,year,nb_grnts,fund_amt,avg_fund_len,avg_team_size,nb_pubs,citations_per_pub,team_size,nb_collabs,nb_collab_countries,nb_pubs_in_topic,pct_pubs_in_topic,first_name,last_name,rsr_gender,rsr_career_start_year,rsr_affiliation,rsr_affiliation_id,rsr_country,rsr_city,orcid_confirmed,rsr_nb_early_pubs,rsr_nb_early_citations,cso_0,cso_1,cso_10,cso_11,cso_12,cso_13,cso_14,cso_15,cso_16,cso_17,cso_18,cso_19,cso_2,cso_20,cso_21,cso_22,cso_23,...,rcdc_57,rcdc_58,rcdc_59,rcdc_6,rcdc_60,rcdc_61,rcdc_62,rcdc_63,rcdc_64,rcdc_65,rcdc_66,rcdc_67,rcdc_68,rcdc_69,rcdc_7,rcdc_70,rcdc_71,rcdc_72,rcdc_73,rcdc_74,rcdc_75,rcdc_76,rcdc_77,rcdc_78,rcdc_79,rcdc_8,rcdc_80,rcdc_81,rcdc_82,rcdc_83,rcdc_84,rcdc_85,rcdc_86,rcdc_87,rcdc_88,rcdc_89,rcdc_9,rcdc_90,rcdc_91,rcdc_92,rcdc_93,rcdc_94,rcdc_95,rcdc_96,rcdc_97,rcdc_98,rcdc_99,inca_prob,weight,rsr_career_age
count,242011,242011,242011,242011,241483,2.167770e+05,241483.000000,242011.000000,242011.000000,242011,242011.000000,242011.000000,6.476800e+04,70321.000000,70620.000000,242011.000000,183500.000000,183500.000000,242011.000000,242011.000000,183500.000000,183500.000000,241505,242011,242011,242011.000000,211761,170643,170643,170643,242011,207636.000000,207636.000000,924.000000,13420.000000,374.000000,12001.000000,44.000000,20636.000000,5027.000000,5665.000000,2189.000000,3828.000000,45584.000000,4763.000000,19250.000000,10087.000000,2805.000000,957.000000,1067.000000,...,495.000000,825.000000,2992.000000,33.0,1991.000000,3201.000000,9042.000000,26169.000000,286.000000,396.000000,88.000000,2332.000000,407.0,11.0,605.000000,341.000000,121.000000,7414.000000,110.000000,176.000000,220.000000,3652.000000,2090.000000,22.0,22.000000,605.000000,55.0,66.0,22.0,627.000000,55.000000,2343.000000,3322.000000,66770.000000,66.0,11.0,319.000000,110.0,5093.000000,5929.000000,2332.000000,23276.000000,1661.000000,693.000000,440.000000,8239.000000,253.000000,242011.000000,242011.000000,242011.000000
unique,5,16925,14004,1171,1082,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7683,9132,3,NaN,4030,1709,60,727,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,National Cancer Institute,grant.2438876,ur.01117731572.33,2009-01-01 00:00:00,2017-01-01 00:00:00,NaN,NaN,NaN,NaN,pre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,David,HEIMBROOK,M,NaN,National Cancer Institute,grid.48336.3a,US,Cambridge,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,161502,495,4224,17743,9108,NaN,NaN,NaN,NaN,132006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6545,4224,139733,NaN,4180,4180,110781,7337,238975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,2007-01-01 00:00:00,2007-01-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2012-12-31 00:00:00,2024-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [49]:
df.to_csv('../data/regression_dataset.csv', index=False)

## Sandbox

### Subsequent Publication RCDC Codes

### Create Lookup Tables

In [ ]:
# grnts_rcdc_1 = grnts_rcdc.drop_duplicates('grant_id', keep='first').reset_index(drop=True)
# pubs_rcdc_1 = pubs_rcdc.drop_duplicates('pub_id', keep='first').reset_index(drop=True)

### Method 1: 1-to-1

In [ ]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# rcdc_comp = pubs_mrg[cols].copy()

In [ ]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc_1, how='left', on='grant_id', suffixes=('', '_1_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_1_grnt', 'rcdc_code': 'rcdc_code_1_grnt'}, inplace=True)

In [ ]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_1_grnt'].notnull())&(rcdc_comp['rcdc_name_1_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_1_grnt'] = rcdc_comp['rcdc_name_1_grnt'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [ ]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_1_grnt', 'rcdc_name_1_pub'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                    , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                    , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                   }).reset_index()

In [ ]:
# rcdc_comp_agency.head()

In [ ]:
# rcdc_comp_agency.tail()

In [ ]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_1_pubs_1_comp.csv', index=False)

### Method 2: 1 to Many

In [ ]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# cols += ['rcdc_names_pub', 'rcdc_codes_pub']
# rcdc_comp = pubs_mrg[cols].copy()

In [ ]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc_1, how='left', on='grant_id', suffixes=('', '_1_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_1_grnt', 'rcdc_code': 'rcdc_code_1_grnt'}, inplace=True)

In [ ]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_1_grnt'].notnull())&(rcdc_comp['rcdc_names_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_1_grnt'] = rcdc_comp['rcdc_name_1_grnt'].fillna('')
#     rcdc_comp['rcdc_names_pub'] = rcdc_comp['rcdc_names_pub'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [ ]:
# def regin(df):
#     return bool(re.search(r"(^|; )\b{}\b(; |$)".format(df['rcdc_name_1_grnt']), df['rcdc_names_pub']))
# rcdc_comp['flag'] = rcdc_comp.apply(regin, axis=1)

In [ ]:
# rcdc_comp['rcdc_name_pub_impute'] = np.where(rcdc_comp['flag']==True
#                                              , rcdc_comp['rcdc_name_1_grnt'], rcdc_comp['rcdc_name_1_pub'])

In [ ]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_1_grnt', 'rcdc_name_pub_impute'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                  , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                  , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                 }).reset_index()

In [ ]:
# rcdc_comp_agency.head()

In [ ]:
# rcdc_comp_agency.tail()

In [ ]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_1_pubs_comp.csv', index=False)

### Method 3: Many to Many

In [ ]:
# cols = ['funder_name', 'funding_amount', 'grant_id', 'rsr_id', 'start_date', 'pub_id', 'citations']
# cols += ['rcdc_names_pub', 'rcdc_codes_pub']
# rcdc_comp = pubs_mrg[cols].copy()

In [ ]:
# rcdc_comp = pd.merge(rcdc_comp, funded_grnt_rcdc, how='left', on='grant_id', suffixes=('', '_grnt'))
# rcdc_comp = pd.merge(rcdc_comp, pub_rcdc_1, how='left', on='pub_id', suffixes=('', '_1_pub'))
# rcdc_comp.rename(columns={'rcdc_name': 'rcdc_name_grnt', 'rcdc_code': 'rcdc_code_grnt'}, inplace=True)

In [ ]:
# # Get rid or ignore Null Values?
# process = 'ignore' # or 'get_rid'
# if process == 'get_rid':
#     rcdc_comp = rcdc_1_comp[(rcdc_comp['rcdc_name_grnt'].notnull())&(rcdc_comp['rcdc_names_pub'].notnull())]
# elif process == 'ignore':
#     rcdc_comp['rcdc_name_grnt'] = rcdc_comp['rcdc_name_grnt'].fillna('')
#     rcdc_comp['rcdc_names_pub'] = rcdc_comp['rcdc_names_pub'].fillna('')
#     rcdc_comp['rcdc_name_1_pub'] = rcdc_comp['rcdc_name_1_pub'].fillna('')
# else:
#     print("Don't forget to choose !")

In [ ]:
# def regin(df):
#     return bool(re.search(r"(^|; )\b{}\b(; |$)".format(df['rcdc_name_grnt']), df['rcdc_names_pub']))
# rcdc_comp['flag'] = rcdc_comp.apply(regin, axis=1)

In [ ]:
# rcdc_comp['rcdc_name_pub_impute'] = np.where(rcdc_comp['flag']==True
#                                              , rcdc_comp['rcdc_name_grnt'], rcdc_comp['rcdc_name_1_pub'])

In [ ]:
# rcdc_comp_agency = rcdc_comp.groupby(['funder_name', 'rcdc_name_grnt', 'rcdc_name_pub_impute'])
# rcdc_comp_agency = pd.DataFrame({'nb_obs': rcdc_comp_agency.size()
#                                  , 'nb_grnts': rcdc_comp_agency['grant_id'].nunique()
#                                  , 'nb_pubs': rcdc_comp_agency['pub_id'].nunique()
#                                 }).reset_index()

In [ ]:
# rcdc_comp_agency.head()

In [ ]:
# rcdc_comp_agency.tail()

In [ ]:
# rcdc_comp_agency.columns = [['funder_name', 'grnt_rcdc', 'pub_rcdc', 'nb_grnts', 'nb_obs', 'nb_pubs']]
# rcdc_comp_agency.to_csv('../output/rcdc_grnt_pubs_comp.csv', index=False)

In [142]:
topics = pd.read_csv('../output/top_topics/rcdc_cso.csv')

In [146]:
cols = list(topics)
cols.remove('rcdc_cso')

In [147]:
topics.head()

,rcdc_cso,INCa/INSERM/DGOS pubs,National Cancer Institute pubs,Cancer Research UK pubs,Wellcome Trust pubs,National Health and Medical Research Council pubs
0,RARE DISEASES * 5. Treatment,1194.276082,6193.329827,634.555952,65.260317,739.683818
1,GENETICS * 1. Biology,1070.227904,11255.215064,1962.520869,436.376389,2118.835029
2,HEMATOLOGY * 5. Treatment,539.686364,2291.762929,197.097619,14.706944,303.995993
3,DIGESTIVE DISEASES * 5. Treatment,440.286346,1662.097754,260.311508,37.942208,287.958267
4,BREAST CANCER * 5. Treatment,421.017262,2211.952915,311.876190,34.180556,332.698810


In [148]:
topics['rcdc'] = topics['rcdc_cso'].apply(lambda x: x[:x.find(" * ")]).str.strip()
topics['rcdc'] = topics['rcdc'].str.replace(',', '')
topics['cso'] = topics['rcdc_cso'].apply(lambda x: x[x.find(" * ")+3:]).str.strip()
topics['cso'] = pd.to_numeric(topics['cso'].str[:1])

In [149]:
df = pd.DataFrame()
for col in cols:
    temp = topics[['rcdc_cso', 'rcdc', 'cso', col]].copy()
    temp.rename(columns={col:'pubs'}, inplace=True)
    temp['agency'] = col.replace(" pubs", "")
    
    if df.empty:
        df = temp
    else:
        df = pd.concat([df, temp], sort=False)

In [150]:
df.to_csv('../output/topics_for_regression.csv', index=False)

In [151]:
topics = pd.read_csv('../output/top_topics/rcdc_cso_by_year.csv')

In [152]:
topics.head()

,rcdc_cso,year,INCa/INSERM/DGOS pubs,National Cancer Institute pubs,Cancer Research UK pubs,Wellcome Trust pubs,National Health and Medical Research Council pubs
0,RARE DISEASES * 5. Treatment,2012,194.792460,926.315938,94.288294,6.885714,88.341180
1,RARE DISEASES * 5. Treatment,2013,166.556548,967.433306,97.803968,11.700992,106.141847
2,RARE DISEASES * 5. Treatment,2014,161.196501,904.290132,78.736508,11.892857,102.677590
3,GENETICS * 1. Biology,2012,150.779365,1801.412926,277.023846,75.898810,301.941540
4,GENETICS * 1. Biology,2013,146.466162,1747.826371,251.646825,74.621032,352.482937


In [153]:
cols = list(topics)
cols.remove('rcdc_cso')
cols.remove('year')

In [154]:
topics['rcdc'] = topics['rcdc_cso'].apply(lambda x: x[:x.find(" * ")]).str.strip()
topics['rcdc'] = topics['rcdc'].str.replace(',', '')
topics['cso'] = topics['rcdc_cso'].apply(lambda x: x[x.find(" * ")+3:]).str.strip()
topics['cso'] = pd.to_numeric(topics['cso'].str[:1])

In [155]:
df = pd.DataFrame()
for col in cols:
    temp = topics[['rcdc_cso', 'rcdc', 'cso', 'year', col]].copy()
    temp.rename(columns={col:'pubs'}, inplace=True)
    temp['agency'] = col.replace(" pubs", "")
    
    if df.empty:
        df = temp
    else:
        df = pd.concat([df, temp], sort=False)

In [156]:
df.head()

,rcdc_cso,rcdc,cso,year,pubs,agency
0,RARE DISEASES * 5. Treatment,RARE DISEASES,5,2012,194.792460,INCa/INSERM/DGOS
1,RARE DISEASES * 5. Treatment,RARE DISEASES,5,2013,166.556548,INCa/INSERM/DGOS
2,RARE DISEASES * 5. Treatment,RARE DISEASES,5,2014,161.196501,INCa/INSERM/DGOS
3,GENETICS * 1. Biology,GENETICS,1,2012,150.779365,INCa/INSERM/DGOS
4,GENETICS * 1. Biology,GENETICS,1,2013,146.466162,INCa/INSERM/DGOS


In [157]:
df.to_csv('../output/topics_for_regression_by_year.csv', index=False)